In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.5 MB/s eta 0:00:00


In [2]:
!pip install editdistance --quiet

In [3]:
import os
import openai
import json

In [4]:
# If you have datakey .json document stored openai api key make this line active to run. Otherwise, make comment here and activate the next line by pasting your api key

OPENAI_API_KEY = ''
with open('datakey.json', 'r') as file_to_read:
    json_data = json.load(file_to_read)
    OPENAI_API_KEY = json_data["OPENAI_API_KEY"]

openai.api_key =  OPENAI_API_KEY

In [5]:
# Make active this to run if you just have api key instead 

#openai.api_key =  "Your api key"

In [48]:
# Import necessary libraries and modules

import re

import random
import time
import threading

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from collections import Counter

!pip install zemberek-python
from zemberek import TurkishMorphology
from collections import OrderedDict
morphology = TurkishMorphology.create_with_defaults()

import editdistance

import pandas as pd

from IPython.display import clear_output
import time


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141210 sha256=740e7198d9f2e5bc269577d46399a612ec795d0acb04ac9e63d21cf4b46e7ab4
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built antlr4-python3-runtime


INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 15.38664984703064


2023-06-01 18:28:22,933 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 15.38664984703064



In [7]:
# Installing and Importing Wordnet

!pip install git+https://github.com/starlangsoftware/TurkishWordNet-Py

from WordNet.WordNet import WordNet

a = WordNet()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/starlangsoftware/TurkishWordNet-Py to /tmp/pip-req-build-o5hep4ua
  Running command git clone --filter=blob:none --quiet https://github.com/starlangsoftware/TurkishWordNet-Py /tmp/pip-req-build-o5hep4ua
  Resolved https://github.com/starlangsoftware/TurkishWordNet-Py to commit 3f7d82bba5918ac4efa826efaeb6921d730bb7ba
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for NlpToolkit-WordNet: filename=NlpToolkit_WordNet-1.0.24-py3-none-any.whl size=11656509 sha256=be0df9622d0e8a53ed0000cce73f52a1802

In [8]:

# Specify the raw URL of the CSV file on GitHub
url = 'https://raw.githubusercontent.com/selimfirat/bilkent-turkish-writings-dataset/master/data/texts.csv'

# Read the CSV file into a Pandas DataFrame
corpus = pd.read_csv(url)

In [9]:
# GPT Model Selection # Note if the current model is overloaded with requests, please change the other one

model_type = "gpt-3.5-turbo" #1

#model_type = "gpt-3.5-turbo-0301" #2


In [10]:
# Head

def Head():
  head = """

  _        _______          _________     _______           _______ _________
  ( \      (  ____ \|\     /|\__   __/    (  ____ \|\     /|(  ___  )\__   __/
  | (      | (    \/( \   / )   ) (       | (    \/| )   ( || (   ) |   ) (   
  | |      | (__     \ (_) /    | | _____ | |      | (___) || (___) |   | |   
  | |      |  __)     ) _ (     | |(_____)| |      |  ___  ||  ___  |   | |   
  | |      | (       / ( ) \    | |       | |      | (   ) || (   ) |   | |   
  | (____/\| (____/\( /   \ )___) (___    | (____/\| )   ( || )   ( |   | |   
  (_______/(_______/|/     \|\_______/    (_______/|/     \||/     \|   )_(   



  """
  check = 0
  while True:
      time.sleep(0.01)
      print(head)
      time.sleep(0.5)
      clear_output()
      check += 1
      if check == 10:
        print(head)
        break


In [11]:
# Instructions and start info. # ALDIK
def StartCommand():
    instructions = """
                                                                LEXI-CHAT Kelime Oyununa Hoşgeldiniz.

    LEXI-CHAT, basit doğal dil işleme teknikleriyle ve doğal dil işleme modeli yapay zeka Chatgpt-3.5 Turbo ile desteklenmiş bir kelime tahmin etme oyunudur.

    Oyuna başlamadan önce lütfen kuralları ve oyun sürecinde, oyundan keyif almanızı sağlayacak adımları içeren aşağıdaki maddeleri okuyunuz.

    1- Bu oyun bir televizyon programı olan "Kelime Oyunu"ndan ilham alarak hazırlanmış ve size sunucunun varlığını hissettirecek şekilde tasarlanmıştır.
    2- Oyunda, size verilen 5 dakikayı kullanarak bilmeniz gereken 4 harflilerden 10 harfli kelimelere kadar her birinden iki tane olacak şekilde toplam 14 soru bulunmaktadır.
    3- Her bir harf değeri 100 puan olmakla beraber oyundan alabileceğiniz maksimum puan  9800 puandır.
    4- Oyunda, herhangi bir kelime için "h" yazarak harf talep edebilirsiniz, her açılan harf için o kelimenin size kazandıracağı puandan 100 puan eksilir.
    5- Oyun süresince size verilen mesaj kutucuğundan, soru öncesi ya da sırasında kelime ile ilgili ek bilgi talep edebilir ve tahminlerinizi yazabilirsiniz.
    6- Yazdığınız tahminlere ya da içinde bulunduğunuz puan durumuna göre sunucu size ek ipuçları verebilir.
    7- Tahminlerinizden birinin doğru olduğunu düşünüyorsanız yanıtınız kabul edilmesi için önce "bb" yazıp akan süreyi durdurmalı ve açılan yeni kutucukta yanıtınızı belirtmelisiniz.
    8- 14 soru da bittikten sonra ya da oyun süresinin tamamını sorulardan önce bitirdiğinizde oyun sonlanacaktır ve toplam puanınız gösterilecektir.

    NOT: Oyundan yüksek verim almak için lütfen oyun sunucusunu aldatmaya yönelik iletişim kurmaktan çekininiz. Evet, siz sunucuyu kandırabilir, hata vermesine neden olabilir ve kırabilirsiniz ancak
    lütfen bir gün onun da sizi kırabileceğini hatırlayın.

                                                                            KEYİFLİ OYUNLAR
    """

    startcommand = """

                                                  Oyunu başlatmak için bölmeye "Ve kayıt" yazabilirsiniz.

    """





    print(instructions)
    time.sleep(4)
    print("\r", startcommand, end="")
    
    while True:
      global start_com
      start_com = input("->: ")
      if start_com.strip() in ["Ve kayıt","ve kayıt"]:
        break
      else:
        print("Henüz oyunda değilsiniz, başlamak için \"Ve kayıt\" yazınız.")

    name = input("Lütfen isminizi girin: ")
    address = input(f"Size nasıl hitap etmemi istersiniz? {name} (hanım/bey): ")
    global username
    username = name + " " + address.lower()

    


In [12]:
# Basic input tokenizer

def input_tokenizer(text):
  return nltk.word_tokenize(text)

In [13]:
# Basic punctiation cleaner

def clean_string(text):
  pattern = re.compile(r'[^\w\s]')
  return re.sub(pattern, "", text)


In [14]:
# GPT | Function for the origin of the words -GPT included

def origin_of_the_words(word_list):
  """This function takes a list of words and returns their origin as a list with the same indexes"""

   # Prompt given to Chatgpt for the origin of the words. This will be computed before the game start.

  messages=[
      {"role": "system", "content": "Sana verilen kelimenin etimolojik kökenini yalnızca yalnızca iki kelime kullanarak söyleyen bir asistansın, bilgiyi etimoloji.com'dan al."},
      {"role": "user", "content": "Bundan sonra vereceğim kelimeler için yanıtları yalnızca örnek formatta ver, başka ekleme yapma (Örnek: \"Arapça kökenli\")."},
  ]

  not_cleaned_origin_list = []
  origin_list = []
  for item in word_list:
    text=item
    message={"role":"user","content":text}
    messages.append(message)
    response = openai.ChatCompletion.create( model=f"{model_type}", messages=messages, max_tokens=7 )
    resp = response["choices"][0]["message"]["content"]
    no_punc = clean_string(resp)
    cleaned = resp.lower().replace(item, "")
    not_cleaned_origin_list.append(cleaned)
    messages.append(response["choices"][0]["message"])

  for phrase in not_cleaned_origin_list:
      match = re.search(r'(.*)\s?kökenli', phrase)
      if match:
          origin_list.append(match.group(1).strip().capitalize()+ " kökenli")
      else:
          origin_list.append("None")

  return origin_list



In [15]:
# GPT | Function for the structure info of the words | Compound or Individual -GPT included

def Structure_of_Word(word_list):
  """This function takes a list of words and returns their structure info as a list with the same indexes"""

   # Prompt given to Chatgpt for the structure info of the words. This will be computed before the game start.

  messages=[
      {"role": "system", "content": "Verilen kelimenin birleşik kelime mi tek kelime mi olduğunu söyle. Yalnızca bilgiyi \"Birleşik kelime\" ya da \"Tek kelime\" olarak ver, başka bir şey yazma."},
      {"role": "user", "content": "sıradağ"},
  ]

  not_cleaned_structure_list = []
  
  for item in word_list:
    text=item
    message={"role":"user","content":text}
    messages.append(message)
    response = openai.ChatCompletion.create( model=f"{model_type}", messages=messages, max_tokens=10 )
    resp = response["choices"][0]["message"]["content"]
    no_punc = clean_string(resp)
    cleaned = resp.lower().replace(item, "")
    not_cleaned_structure_list.append(cleaned)
    messages.append(response["choices"][0]["message"])

  if len(not_cleaned_structure_list) > 14:
    del not_cleaned_structure_list[0]
  filtered_list = [re.findall(r'birleşik kelime|tek kelime', i) for i in not_cleaned_structure_list]

  structure_list = [i[0] for i in filtered_list]

  return structure_list



In [16]:
# This GPT Prologue function. Makes a short conversation before starting and asks user whether they are ready.

def Gpt_Prologue():
  print(f"Merhaba {username}, Lexichat'e hoşgeldiniz!")

  messages=[
      {"role": "system", "content": "Sen Lexichat adlı bir Kelime Oyunu yarışması sunucususun Yarışma öncesi kısa bir sohbet gerçekleştirmek senin amacın. Önemli olan, diyaloğu yarışmacıyı rahatlatmak için yapman. Daha çok sohbet ağzı benimse ancak mutlaka \"siz\" diye hitap et. Birkaç karşılıklı diyalog gerçekletikten sonra yarışma için hazır olup olmadığını soracaksın."},
      {"role": "user", "content": "Merhaba"},
  ]
  response = openai.ChatCompletion.create( model=f"{model_type}", messages=messages, max_tokens=100 )
  print(response["choices"][0]["message"]["content"])

  print("Bu arada hazır olduğunuzda her zaman \"hazırım\" yazabilirsiniz.")
  tokenized_inp = []
  while True:
    text=input("--> ")
    
    if "hazırım" in input_tokenizer(text.lower()) or "evet hazırım" in input_tokenizer(text):
      print("Öyleyse başlayabiliriz... Şöyle derin bir nefes alın...")
      break
    message={"role":"user","content":text}
    messages.append(message)
    response = openai.ChatCompletion.create( model=f"{model_type}", messages=messages, max_tokens=100 )
    print(response["choices"][0]["message"]["content"])
    messages.append(response["choices"][0]["message"])

  time.sleep(2)
  print("Nefesi aldınız mı..?")
  time.sleep(0.5)
  print("Artık başlamaya hazırsınız sanırım?")
  text = input("--> ")
  if text in ["hazırım", "hazır", "başla", "basla", "evet"]:
    print("İlk soruyla başlıyorum öyleyse...\n")

In [17]:
# Function for word function (V, N, ADV, etc.)
def function_of_word(word_list):
  function_list = ["None"]*14
  for i in range(len(word_list)):
    if get_POS(word_list[i]) != "None":
      function_list[i] = get_POS(word_list[i]) 
  return function_list


In [18]:
# Letter request function

revealed_letters = []
def LetterRequest(word):
  import random

  word = word.strip()
  word_sliced = [None] * len(word)

  for i in range(0, len(word)):
    word_sliced[i] = word[i]
    if len(revealed_letters) != len(word):
      revealed_letters.append("_  ")

  random_num = random.randint(0, len(word)-1)
  if revealed_letters[random_num] == "_  ":
    revealed_letters[random_num] = word_sliced[random_num]
  elif "_  " not in revealed_letters:
    print("Efendim, tüm harfleri açtınız, artık bir sonraki soruya...\nHarf alamazsınız artık.")
    
  else:
    LetterRequest(word)
  return revealed_letters

In [71]:
# Round_Time function

def Round_Time(round):
  global active_input_bb
  active_input_bb = ""
  countdown_time = 45
  check_ = 0
  
  def check_input():
      print("")
      chance_list = [2, 3, 4, 7, 9, 5, 25, 65]
      list_active_input_bb = []
      while True:
        if check_ == 1:
          break
        else:
          global active_input_bb
          active_input_bb = input("-BB> ")
          tkn_input_bb = input_tokenizer(active_input_bb.lower())
          if active_input_bb.lower().strip() == word:
            if chosen_phrase == "Bana mı soruyorsunuz, cevap mı veriyorsunuz?..":
              phrases = ["Cevap veriyorlar..!", "Sanırım cevap veriyorsunuz ve doğru olanı yapıyorsunuz"]
              print("\n",phrases[random.randint(0, (len(phrases)-1))]) 
            elif chosen_phrase == f"{word} sığıyor mu oraya?":
              print("\nEyvah, eyvah! Efendim sığıyor mu ki?...")
              time.sleep(3)
              print("Tabii ki sığıyor! Yalnızca biraz heyecanlandırmak istedim ancak buna kanmadılar kendileri...") 
            
            round_score = (LetterRequest(word).count("_  ") + 1) * 100
            congrats_phrases = [f"Tebrik ederim efendim {word} doğru cevap ve size {round_score} puan kazandırıyor...", f"Bir {round_score} puan geliyor...", f"Bu soruyla birlikte toplam puanınıza bir {round_score} puan daha eklediniz...", f"{round_score} puan cepte", f"Tertemiz bir {round_score} puan...", f"{round_score} puanı kasamıza ekliyoruz...", f"{word}, size {round_score} kazandırıyor"]
            print("\n",congrats_phrases[random.randint(0,(len(congrats_phrases) - 1))])
            global total_score
            total_score += round_score
            break
          elif (active_input_bb.lower().strip() == "h") or ("harf" in tkn_input_bb and ("alabilir" in tkn_input_bb or "alayım" in tkn_input_bb)):
            print("\nEfendim, harf alamazsınız artık, süreyi durdurdunuz.")

          elif any(item in ["kök", "köken", "kökenli", "kökeni"] for item in tkn_input_bb):
            if Origin_list[round] != "None":             
              origin_phrases = [f"Sanırım {Origin_list[round]} olmalı", f"Hmm... Bu, sanıyorum {Origin_list[round]}", f"{Origin_list[round]}", f"{Origin_list[round]} olma ihtimali yüksek"]
              print("\n",origin_phrases[random.randint(0, (len(origin_phrases) - 1))]) 
            else:
              print("\nMaalesef kökeninden emin değilim...")

          elif any(item in ["yardım", "yardımcı", "ipucu", "destek", "bulamadım", "bilemedim"] for item in tkn_input_bb):

            if len(chance_list) == 0:
              negative_phrases = ["Maalesef, daha fazla yardımcı olamam...", "Efendim, daha ne diyeyim ki... Bulursunuz siz bunu...", "Başka bir şey gelmiyor ki aklıma...", "Maalesef..! Odaklanın, bulursunuz...\nSoruyu bir daha okuyun isterseniz..."]
              print("\n",negative_phrases[random.randint(0, (len(negative_phrases)))]) 
            else:
              chance_num = random.choice(chance_list)
              
              if chance_num % 3 == 0:
                print("\nÖrnek cümle verebilirim belki...")
                time.sleep(1)
                print("\n",Example_sentences[round])
                chance_list.remove(3)
                chance_list.remove(9)

              elif chance_num % 5 == 0:
                if Compound_list[round] != "None":
                  print(f"\nEfendim buna dikkat... {Compound_list[round]}")
                elif Additional_defs[round] != None:
                  print(f"\nŞöyle de tanımlanabilir...\n{Additional_defs[round]}")
                else:
                  print(f"\nBiraz daha düşünün efendim, tekrar okuyun soruyu...\n{definition_list[round]}")

                chance_list.remove(5)
                chance_list.remove(25)
                chance_list.remove(65)

              elif chance_num % 2 == 0:
                print("\nHmm...")
                time.sleep(1)
                try:
                  print("\n",getSynonym(round))
                except:
                  print("\n",get_example_sentence(word))
                chance_list.remove(2)
                chance_list.remove(4)
                time.sleep(1)
                print("\nDüşünün biraz daha...")

              elif chance_num % 7 == 0:
                print("\nHmm... Belki bu yardımcı olabilir...")
                time.sleep(1)
                print("\n",Example_sentences[round])
                time.sleep(2)
                print("Ayrıca...")
                try:
                  getSynonym(round)  
                except:
                  get_example_sentence(word)
                chance_list.remove(7)
                print("\nOdaklanırsanız bulursunuz bence... Tekrar okuyun...")

          else:
              string_familiarity = [similar_word_hint(word, 3)]
              list_synonym = [synonym_hint(word)]
              #return string_familiarity
              #return list_synonym

              if active_input_bb.lower().strip() not in list_active_input_bb:
                string_familiarity = similar_word_hint(word, 3)
                list_synonym = synonym_hint(word)
                  
              if active_input_bb.lower().strip() in string_familiarity and active_input_bb.lower().strip() in list_synonym:
                print("\nHadi bir daha, çok çok yaklaştınız...")

              elif active_input_bb.lower().strip() in string_familiarity or any(item in string_familiarity for item in tkn_input_bb):
                print("\nÇok yaklaştınız, hadi birkaç harf değiştirin.")

              elif active_input_bb.lower() == "":
                phrases_list_empty = ["Bir harf alabilirsiniz belki...", "...", "Zamana dikkat!", "Zaman akıyor efendim, bir harf mi istesek..?", "..."]
                print("\n",phrases_list_empty[random.randint(0, (len(phrases_list_else)-1))])              
              
              elif active_input_bb.lower().strip() in list_synonym or any(item in list_synonym for item in tkn_input_bb):
                synonym_phrases = ["Bu değil efendim, öteki... Eş anlamlısı lazım bize...", "Hadi bir daha deneyin... Bu sefer eş anlamlısını söyleyin...", f"{active_input_bb} değil de... Bir benzeri...\nNeydi o?\nBize eş anlamlısı lazım...", "Yaklaştınız... Aynı anlama gelen başka kelime daha var...\nNeydi o..?"]
                print("\n",synonym_phrases[random.randint(0, (len(synonym_phrases) - 1))])

              elif active_input_bb.lower().strip() in list_active_input_bb or any(item in list_active_input_bb for item in tkn_input_bb):
                print("\nBunu zaten söylediniz, tekrar düşünün...")         

              else: 
                if editdistance.distance(active_input_bb.lower().strip(), word) <= 2:
                  if round >= 6:
                    distance_phrases = ["Çok yaklaştınız, acaba bir iki harf mi değiştirsek..?", f"{active_input_bb.lower()} doğru muydu yoksa birkaç harf mi farklıydı?"] 
                    print(distance_phrases[random.randint(0, len(distance_phrases) - 1)])   
                else:
                  phrases_list_else = ["Süre akıyor, tekrar deneyin...", "Biraz daha düşünün isterseniz...", "Ah, ah! Keşke bir harf daha alsaydınız...", "Zamana dikkat! Zamana dikkat..!", "Değil, değil... Zaman akıyor efendim...", "Hayır, hayır... Dikkatlice tekrar okuyun isterseniz soruyu..."]
                  print("\n",phrases_list_else[random.randint(0, (len(phrases_list_else)-1))])
            
      list_active_input_bb.append(active_input_bb)
      chance_list = [2, 3, 4, 7, 9, 5, 25, 65]
      revealed_letters = []

  

    

  input_thread = threading.Thread(target=check_input)
  input_thread.start()

  # Start the countdown
  for i in range(countdown_time, 0, -1):
    print("\r", i, end="")  # Use '\r' to overwrite the previous line
    time.sleep(1)  # Wait for 1 second
    if word.lower().strip() in active_input_bb.lower().strip().split():
      break
    if i == 1:
      print("\n")
      print("\nÜzgünüm süreniz bitti, bakalım neymiş yanıt...\n")
      time.sleep(1.5)
      print(word_list[round])
      round_score = (LetterRequest(word).count("_  ") + 1) * 100
      global total_score
      total_score -= round_score
      negative_score_phrases = [f"Bu soru size {round_score} puan kaybettirdi... Ama olsun toparlayabiliriz hâlâ...", f"Ah, ah... {total_score} puana geriledik...\Moral bozmak yok devam edelim..."]
      print(negative_score_phrases[random.randint(0, len(negative_score_phrases) - 1)])
      check_ = 1
      break
  input_thread.join()

In [20]:
# GPT | Function for the example sentences of the words - GPT included
 
def example_sentence(word_list):
  """This function takes a list of words and returns their 3 usage in a sentence as a list with the same indexes"""

   # Prompt given to Chatgpt for creating example sentences of the words. This will be computed before the game start.

  messages=[
      {"role": "system", "content": "Sana verilen kelime ile basit örnek bir cümle yaz verilen kelime yerine _______ koy. 3 kelimeden oluşsun."},
      {"role": "user", "content": "İlk kelime geliyor"},
  ]
  
  sentence_list = []

  for item in word_list:
    text=item
    message={"role":"user","content":text}
    messages.append(message)
    response = openai.ChatCompletion.create( model=f"{model_type}", messages=messages, max_tokens=200 )
    
    resp = (response["choices"][0]["message"]["content"])
    no_punc = clean_string(resp)
    gg = resp.lower().replace(item, "-------")
    sentence_list.append(gg)
    messages.append(response["choices"][0]["message"])

  return sentence_list



In [21]:
# Pre_info function, information just before the next question --> Works, can be developed root info can be taken when demanded

def Pre_info(question_number, word, total_score, round):
  """ This function takes three inputs (question number, word, and total_score at that round), and returns phrases regarding these. This code requires random and openai libraries to run properly"""
  import random
  num = question_number
  letter_count = (((question_number-1) // 2) + 4)
  question_point = letter_count* 100
  
  # Prepared phrases for info
  list_of_info_1 = [f"{letter_count} harfli sorular ile devam ediyoruz efendim...", f"{question_number}. sorudayız...", f"{question_number}. soruya geldik..."]
  list_of_info_2 = [f"Şu an {total_score} puandasınız, bu soru ile {total_score + question_point} puana ulaşabilirsiniz...", f"Sıradaki soru ile {question_point} puan sizi bekliyor efendim...", f"Sıradaki soru {question_point} puan değerinde..."]
  mumbling = ["Hmm...", "Bakalım...", "Tamam...", "Şimdi bakalım...", "Şöyle ki...", "Hıh...", "Peki..."]
  list_of_info_first_q = ["Tertemiz bir 400 puanla başlayabiliriz efendim...", "Güzel bir başlangıç yaparız umarım bu soruyla, dur bakalım...", "Şöyle silkinelim ve ilk soruyla başlayalım..."]
  
  # Info for function of word
  f_type = Function_list[round] 
  function_phrases = [f"Bu kelime bir {f_type}...", f"Sıradaki kelime bir {f_type}...", f"Bu bir {f_type}..."]

  # Info for structure of word
  s_type = Structure_list[round]
  structure_phrases = [f"Hmm, {s_type}den oluşan bir sözcük ile devam ediyoruz...", f"Sıradaki bir {s_type}, görelim neymiş...", f"Bu bir {s_type}... Bakalım neymiş...", f"Sonraki sözcük, {s_type}den oluşmakta..."]

  # Origin info        
  output_root = Origin_list[round] 
  question_info = [f"Sıradaki kelime {output_root}", f"Bu kelime sanırım {output_root}.. Dur bakalım çıkarabilecek misiniz...", f"Kelimenin kökenine baktığımda...\nhmm sanırım {output_root} olmalı", f"Bu kelime, dilimize {output_root} bir kelimeden yerleşmiş."]

  # Indexing for prepared phrases lists
  index_info1 = random.randint(0, (len(list_of_info_1)-1))
  index_info2 = random.randint(0, (len(list_of_info_2)-1))
  index_mumbling = random.randint(0, (len(mumbling)-1))
  index_question_info = random.randint(0, (len(question_info)-1))
  index_info_first_q = random.randint(0, (len(list_of_info_first_q)-1))
  index_f_type = random.randint(0, (len(function_phrases)-1))
  index_s_type = random.randint(0, (len(structure_phrases)-1))
  
  # Info for the first question
  if question_number == 1:
    print("\n",list_of_info_first_q[index_info_first_q],"\n")
    time.sleep(0.5)
    if f_type != "None":
      print(f"\nBu kelime bir {f_type}...")
    time.sleep(0.5)
    print("\n",mumbling[index_mumbling],"\n")
    time.sleep(1)
    if s_type != "None":
      print(f"\nBu bir {s_type}... Bakalım neymiş...")
  
  # Info other than the first question
  else:
    print("\n",list_of_info_1[index_info1])
    time.sleep(1)
    print("\n",list_of_info_2[index_info2],"\n")
    time.sleep(1.2)
    print("\n",mumbling[index_mumbling],"\n")
    time.sleep(1)
    if f_type != "None":
      print("\n",function_phrases[index_f_type])
  
    # Chance of give origin info
    hint_chance = random.randint(1,5)
    if hint_chance % 4 == 0:
      if output_root != "None":    
        print("\n",question_info[index_question_info])
    time.sleep(1)
    if hint_chance % 2 == 0:
      if f_type != "None":   
        print("\n",structure_phrases[index_s_type])
    time.sleep(1)

In [22]:
# Function for Synonym | Returns synonym list

def synonym_hint(word):
  word_synset = a.getSynSetWithLiteral(word.lower(), 1)
  try:
    synonyms = str(word_synset.getSynonym())
    synonym = re.sub(r'â', 'a', synonyms)
    lists_synonym = synonym.split()
    for x in lists_synonym:
      if x == word:
        lists_synonym.remove(x)
    if len(lists_synonym) != 0:
      return lists_synonym
    else: 
      return "None"

  except:
    return "None"




In [23]:
# Function for getting synonyms | We will create the synonym_list 

def getSynonym(round):
    synonym_list_copy = synonym_list.copy()  # Create a copy of the list
  
    if synonym_list_copy[round] == "None":
        return "None"  # Return None if the synonym list is empty
    num_synonyms = min(3, len(synonym_list_copy[round]))  # Determine the number of available synonyms
    synonyms = random.sample(synonym_list_copy[round], num_synonyms)  # Select random synonyms
    result = [f"Belki bunlar size yardımcı olabilir: {', '.join(synonyms)}...\nBu kelimeler eş anlamlı olarak kullanılır diyebilirim...", f"{', '.join(synonyms)}... Bunlara dikkat! Yardımcı olabilir...", f"{', '.join(synonyms)}... Bu kelimeler bir şeyler anımsatabilir...", f"{', '.join(synonyms)}... Bu kelimeleri düşünün..."]
    return result[random.randint(0, (len(result)-1))]  # Return the synonyms as a string separated by commas




In [24]:
# Bilkent dataset and Function for edit distance

#preprocessing of the bilkent dataset

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

cor = corpus.to_string(index=False)

cor = cor.replace('\n',"")
#cor = re.sub("<[^\<]>", "", str(corpus))
cor = re.sub("<[^\<]Wd>", "", str(corpus))

cor = cor.lower()

tokens = nltk.word_tokenize(cor)

cleaned_tokens = []

for word in tokens:
  if not word.isnumeric() == True and not word.isalpha() != True and not len(word) <= 1:
    cleaned_tokens.append(word)


word_count = Counter(cleaned_tokens)
sorted_keywords = list(sorted(word_count.items(), key=lambda x: x[1], reverse=True))

#edit distance

def similar_word_hint(word, n_suggestions):
    
    if word in cleaned_tokens:
      return word
    else:

      distance = []
      for token in cleaned_tokens:
        d = editdistance.distance(word, token)
        distance.append(d)

      min_dist = min(distance)

      candidates = {}
      for x,y in sorted_keywords:
        candidates[x] = editdistance.distance(word, x)
        
      suggestions = []
      for candidate, dis in candidates.items():
        if min_dist - 1 < dis <= min_dist:
          if len(candidate) == len(word):
            suggestions.append(candidate)
            if len(suggestions) == n_suggestions:
              return suggestions
            
      return suggestions

In [25]:
# This is for getting functions of verbs

def get_POS(word):
  pos_dict = {'ADJECTIVE': 'sıfat', 'VERB': 'fiil', 'ADVERB': 'zarf', 'NOUN': 'isim', 'INTERJECTION': 'nida/ünlem', 'CONJUNCTION': 'bağlaç', 'PREPOSTION': 'edat', 'PRONOUN': 'zamir'}
  word_synset = a.getSynSetWithLiteral(word.lower(), 1)
  try:
    pos = str(word_synset.getPos())
    pos = re.sub('Pos.', '', pos)
    return pos_dict[pos]
  except:
    return "None"

  

In [26]:
# This is for combining two example sentence lists

def merge_lists(list1, list2):  
    merged_list = []
    for i in range(len(list1)):
        if list1[i] != "None":
            merged_list.append(list1[i])
        elif list2[i] != "None":
            merged_list.append(list2[i])
    return merged_list


In [27]:
# Next 9 cells gets words, definitions, synonyms, example sentences, functions; and must be run in order (1)

def select_words_and_meanings(file_list, word_lengths):
    words = []
    meanings = []
    selected_words = []
    selected_meanings = []
    selected_meanings2 = []
    selected_meanings3 = []
    
    for file in file_list:
        with open(file, 'r', encoding='utf-8') as file_object:
            lines = file_object.readlines()
            for line in lines:
                word, meaning = line.strip().split(": ")                  
                if len(word) in word_lengths:
                    words.append(word)
                    meanings.append(meaning)
              
    for word_length in word_lengths:
        words_with_length = [word for word in words if len(word) == word_length]
        selected_words.extend(random.sample(words_with_length, 2))
    
    for word in selected_words:
        indices = [i for i, x in enumerate(words) if x == word]
        if len(indices) == 1:
            selected_meanings.append(meanings[indices[0]])
            selected_meanings2.append(None)
            selected_meanings3.append(None)
        elif len(indices) >= 2:
            selected_meanings.append(meanings[indices[0]])
            selected_meanings2.append(meanings[indices[1]])
            if len(indices) >= 3:
                selected_meanings3.append(meanings[indices[2]])
            else:
                selected_meanings3.append(None)

    return selected_words, selected_meanings, selected_meanings2, selected_meanings3
  

In [28]:
# (2) Correcting Turkish character confusion 

class UnicodeTr(str):  #NEW
    CHARMAP = {
        "to_upper": {
            "ı": "I",
            "i": "İ",
        },
        "to_lower": {
            "I": "ı",
            "İ": "i",
        }
    }

    def lower(self):
        for key, value in self.CHARMAP.get("to_lower").items():
            self = self.replace(key, value)
        return self.lower()

def apply_unicode_transform(selected_words):
    transformed_words = []
    for word in selected_words:
        transformed_word = UnicodeTr(word).lower()
        transformed_words.append(transformed_word)
    return transformed_words


In [29]:
# (3) | Compound_function

def compound_function(transformed_words):
    with open('/content/gts (1).json', 'r') as file:
        data = json.load(file)
        compound_list = []
        
        for word in selected_words:
            found_example = False
            for entry in data:
                if entry['madde'].lower() == word.lower():
                    if entry.get("birlesikler"):
                        birlesikler = entry["birlesikler"]
                        birlesikler_listesi = birlesikler.split(", ")
                        for birlesik in birlesikler_listesi:
                            if word.lower() in birlesik.lower():
                                pattern = r'\b' + re.escape(word) + r'(\w*)\b'
                                match = re.search(pattern, birlesik, flags=re.IGNORECASE)
                                if match:
                                    suffix = match.group(1)
                                    new_birlesik = re.sub(pattern, "______" + suffix, birlesik, flags=re.IGNORECASE)
                                    compound_list.append(new_birlesik)
                                    found_example = True
                                    break
                        if found_example:
                            break
            if not found_example:
                compound_list.append("None")
                
    return compound_list





In [30]:
# (4) Example sentence function (gts.json) | This is combined with gpt examples and assigned to a new list below
def example_sentences_function(selected_words):
    with open('/content/gts (1).json', 'r') as file:
        data = json.load(file)
        
    example_sentence_list = []
    
    for word in selected_words:
        found_example = False
        new_example = ""  # Initialize new_example with a default value
        
        for entry in data:
            if entry['madde'].lower() == word.lower():
                if entry.get("anlamlarListe"):
                    for anlam in entry["anlamlarListe"]:
                        if anlam.get("orneklerListe"):
                            for ornek in anlam["orneklerListe"]:
                                if ornek.get("ornek"):
                                    sentence = ornek["ornek"]
                                    pattern = r'\b' + re.escape(word) + r'(\w*)\b'
                                    match = re.search(pattern, sentence, flags=re.IGNORECASE)
                                    if match:
                                        suffix = match.group(1)
                                        new_example = re.sub(pattern, "______" + suffix, sentence, flags=re.IGNORECASE)
                                        found_example = True
                                        break  # Exit the innermost loop
                            if found_example:
                                break  # Exit the middle loop
                    if found_example:
                        break  # Exit the outermost loop
        if not found_example:
            example_sentence_list.append("None")
        else:
            example_sentence_list.append(new_example)
                        
    return example_sentence_list



In [49]:
# (5.1) Example sentence function requirement

def analyze_word(word):
    results = morphology.analyze(word)
    analyzed_results = []
    for result in results:
        analyzed_results.append(str(result))
    output = re.findall(r'\[(.*?)\]', ', '.join(analyzed_results))
    output = [element.split(":")[0] for element in output]
    output = list(OrderedDict.fromkeys(output))
    output = [item.strip('\'') for item in output]
    return output[0] if output else ""  # Return the first element of output as a string, or an empty string if output is empty

In [72]:
# (5.2) Example sentence function (wordnet) | This is called separately when user requests 

def get_example_sentence(word):
  word_synset = a.getSynSetWithLiteral(word, 1)
  s_hint_list = synonym_hint(word)
  #example_sentences = ''.join(word_synset.getExample())
  new_example = ''
  other_example = ''
  worD = r"\b{}(?:\w+)?\b".format(re.escape(word))#to get the word and possible derivations


  if type(word_synset.getExample()) == type(None):
    new_example = 'None'
    
  

  else:#for if the sentence uses the original word in the example sentence
    example_sentences = ''.join(word_synset.getExample())
    example_sentences = example_sentences.lower()
    example_tokens = word_tokenize(example_sentences)###NEW####
    stem_dict = {}######NEW####
    for x in example_tokens:###NEW####
      stem_dict[x] = analyze_word(x)###NEW####
      if word == stem_dict[x]:###NEW####
        new_example = re.sub(x, "______", example_sentences)###NEW####



    if s_hint_list != []:
      for z in s_hint_list:#for example sentences where a synonyms is used in the sentence ###NEW#### change to "z"
        example_sentences = ''.join(word_synset.getExample())
        example_sentences = example_sentences.lower()
        hint = r"\b{}(?:\w+)?(?=\b)".format(re.escape(z))#the synonym as well as possible derivations
        search = ''.join(re.findall(hint, example_sentences))#to see if the synonyms is used or not in the sentence
        if search != '':
          example_sentences = example_sentences.lower()
          example_sentences = example_sentences.lower()
          example_tokens = word_tokenize(example_sentences)###NEW####
          stem_dict = {}###NEW####
          for y in example_tokens:###NEW####
            stem_dict[y] = analyze_word(y)###NEW####
            if z == stem_dict[y]:###NEW####
              new_example = re.sub(y, lambda match: match.group(0).upper(), example_sentences)#to show the synonym in UPPER letters ###NEW####
              other_example = re.sub(y, "______", example_sentences)#to replace a blank ###NEW####
  
    new_example = new_example + "\nDeğiştirin bu kısmı..." + " Tekrar ediyorum...,\n" + other_example +"..."
        
  return new_example 
  

In [32]:
# (6)
file_list = ['4.letters.txt', '5.letters.txt', '6.letters.txt', '7.letters.txt', '8.letters.txt', '9.letters.txt', '10.letters.txt']
word_lengths = [4, 5, 6, 7, 8, 9, 10]

selected_words, selected_meanings, selected_meanings2, selected_meanings3 = select_words_and_meanings(file_list, word_lengths)


In [35]:
# ###Important### (6.x1) Instead of this if works run the second one  | (This can take a while) If this does not work either, run (6.1) and jump to (7.1) and active proper lists and run it. Then continue with (7.2)


while True:
  try:
    example_gts = example_sentences_function(word_list)
    break
  except:
    selected_words, selected_meanings, selected_meanings2, selected_meanings3 = select_words_and_meanings(file_list, word_lengths)
    word_list = apply_unicode_transform(selected_words)


In [ ]:
# (6.x2) # if this does not work, deactivete this (add "#" each line); and activate and run (6.x1). Then continue from the (6.1) (This may give an error on Colab. Unfortunately, We do not know the reason. )
#word_list = apply_unicode_transform(selected_words)
#example_gts = example_sentences_function(word_list)


In [37]:
# (6.1) | These are necessary lists to be defined


definition_list = []
definition_list2 = []
definition_list3 = []
synonym_list = []
revealed_letters = []

total_score = 0


In [38]:
# (7) | Assigning functions to defined lists
for i in range(0,14):
    definition_list.append(selected_meanings[i])
    definition_list2.append(selected_meanings2[i])
    definition_list3.append(selected_meanings3[i])

In [40]:
# (7.1)

for word in word_list: 
  if synonym_hint(word) != "None":
    synonym_list.append(synonym_hint(word))
  else:
    synonym_list.append("None")



In [ ]:
# (7.2)
#Just in case lists: If it gives error... (This was to be used during presentation)

#word_list = ['post', 'uzay', 'insaf', 'külah', 'kraker', 'dışses', 'babişko', 'gösteri', 'vasistas', 'naftalin', 'uçukkaçık', 'gençirisi', 'çığıraçmak', 'birçırpıda']
#definition_list = ['Şömine önü aksesuarı', 'Bütün gök cisimlerinin içinde bulunduğu sonsuz boşluk', 'Acımaya, vicdana veya mantığa dayanan adalet', 'İçine bir tür tatlı konulan yenilebilir kap', 'Gevrek tuzlu bisküvi', 'Sinema ve televizyonda hikâyeye gaipten katılan anlatıcı', 'Erkek ebeveyne yönelik, kimilerine göre samimiyetin sınırlarını zorlayan bir hitap şekli', 'Bir istek veya karşı görüşün, halkın ilgisini çekecek biçimde topluca ve açıkça yapılması', 'Almanlarla ilgili bir şakadan türemiş ve genelleşerek bir pencere türünün adı haline gelmiş söz', 'Kumaş, elbise, halı gibi eşyaları korumakta kullanılan antiseptik bir hidrokarbon', 'Rastgele ve patavatsız bir biçimde hareket eden, deli dolu', 'Battal ergen', 'Bir alanda yeni bir yol, yöntem başlatmak', '“Ele alır almaz, çabucak, tek seferde” anlamında bir söz dizisi']
#synonym_list = ['None', ['feza', 'mekan', 'mekan', 'feza'], 'None', 'None', 'None', 'None', 'None', ['demonstrasyon'], 'None', 'None', 'None', 'None', 'None', 'None']
#Additional_defs = ['Tilki tilkiliğini bildirinceye kadar ... elden gider', 'Dünya ve atmosferi dışında, evrenin geri kalan bölümü için kullanılan bir söz', 'Vicdana dayalı adalet', 'Konik kâğıt kap', None, None, 'Yüz göz olunmuş erkek ebeveyn unvanı', None, 'Gurbetçi pencere', 'Özellikle güveye karşı kullanılan kokulu bileşik', None, 'Çabuk serpilip büyümüş yaşı küçük kişi', None, None]

In [41]:
# (7.3)

try:
  Example_sentence_list = example_sentence(word_list)
except:
  Example_sentence_list = ["None"]*14 

In [51]:
# (7.4) | Combining example sentence lists 

Example_sentences = merge_lists(example_gts, Example_sentence_list)
Additional_defs = merge_lists(definition_list2, definition_list3)


In [ ]:
# List Check

#print(word_list)
#print(definition_list)
#print(synonym_list)
#print(Additional_defs)

In [43]:
# (8) Assigning origin to Origin_list

try:
  Origin_list = origin_of_the_words(word_list)
  
except:
  Origin_list = ["None"]*14
 



In [44]:
# (8.1) Assigning word functions to Function_list

Function_list = function_of_word(word_list)


In [45]:
# (8.2) Assigning compound hint to Compound_list

Compound_list = compound_function(word_list)

In [54]:
# (8.3) Assigning structure info (V, N, Adj, etc.) to Structure_list

Structure_list = Structure_of_Word(word_list)

In [73]:
# This is the main function

def __main__x():
  def countdown_timer():
      global total_time, is_paused, resume_time
      total_time = 5*60
      while total_time > 0:
          if is_paused:
              time.sleep(1)
              continue

          mins, secs = divmod(total_time, 60)
          timer_display = "{:02d}:{:02d}".format(mins, secs)
          print("\r", "Kalan Zaman:", timer_display, end="")

          time.sleep(1)
          total_time -= 1
          

      print("Maalesef efendim, süre sona erdi.")


  def Process():
      global total_time, is_paused, resume_time, total_score

      global chosen_phrase
      chosen_phrase = "" 
      list_active_input = []
      chance_list = [2, 3, 4, 7, 9, 5, 25, 65]
      print("\n")
      while True:
        active_input = input("\n-> ")
        
        tkn_input = input_tokenizer(active_input.lower())
        
        if active_input.lower().strip() == word.lower().strip() or word in tkn_input:
          phrase_list = [f"{word} sığıyor mu oraya?", "Bana mı soruyorsunuz, cevap mı veriyorsunuz?..", "Haydi, şöyle bir cesaret... ", "Efendim bana sormayın... Ben bir şey diyemem ki...", "Ben bilmem...", "... emin misiniz..?", "Risk alacak mısınız..?" , "Buton... Buton..., Unutuyorsunuz basmayı ya da risk almak mı istemiyorsunuz..?", ""]
          chosen_phrase = phrase_list[random.randint(0, (len(phrase_list)-1))]
          print("\n",chosen_phrase)

        elif any(item in ["kök", "köken", "kökenli", "kökeni"] for item in tkn_input):
          if Origin_list[round] != "None":             
            origin_phrases = [f"Sanırım {Origin_list[round]} olmalı", f"Hmm... Bu, sanıyorum {Origin_list[round]}", f"{Origin_list[round]}", f"{Origin_list[round]} olma ihtimali yüksek"]
            print("\n",origin_phrases[random.randint(0, (len(origin_phrases) - 1))]) 
          else:
            print("\nMaalesef kökeninden emin değilim...")

        elif active_input.lower().strip() == "bb":
          if is_paused:
              is_paused = False
              total_time = resume_time
          else:
              is_paused = True
              resume_time = total_time  
          Round_Time(round)
          global revealed_letters 
          revealed_letters = []
          break
            
        elif (active_input.lower().strip() == "h") or ("harf" in tkn_input and ("alabilir" in tkn_input or "alayım" in tkn_input)) or ("harf" in tkn_input):
          current_letters = " ".join(LetterRequest(word))
          print("\n",current_letters,"\n")
          if "_  " not in current_letters:
            print("\nÜzgünüm, bu sorudan puan alamadınız!\nOlsun! Üzülmeyin, rakiplerinizin de puan kaybetmeyeceği ne malum...\nBiz sıradaki soru ile devam edelim...\n") 
            revealed_letters = []
            break 
        
        elif any(item in ["yardım", "yardımcı", "ipucu", "destek", "bulamadım", "bilemedim"] for item in tkn_input):
            
            if len(chance_list) == 0:
              negative_phrases = ["Maalesef, daha fazla yardımcı olamam...", "Efendim, daha ne diyeyim ki... Bulursunuz siz bunu...", "Başka bir şey gelmiyor ki aklıma...", "Maalesef..! Odaklanın, bulursunuz...\nSoruyu bir daha okuyun isterseniz..."]
              print("\n",negative_phrases[random.randint(0, (len(negative_phrases)))]) 

            else:
              chance_num = random.choice(chance_list)
              
              if chance_num % 3 == 0:
                print("\nÖrnek cümle verebilirim belki...")
                time.sleep(1)
                print(Example_sentences[round])
                chance_list.remove(3)
                chance_list.remove(9)
              
              elif chance_num % 5 == 0:
                if Compound_list[round] != "None":
                  print(f"\nEfendim buna dikkat... {Compound_list[round]}")
                elif Additional_defs[round] != None:
                  print(f"\nŞöyle de tanımlanabilir...\n{Additional_defs[round]}")
                else:
                  print(f"\nBiraz daha düşünün efendim, tekrar okuyun soruyu...\n{definition_list[round]}")

                chance_list.remove(5)
                chance_list.remove(25)
                chance_list.remove(65)


              elif chance_num % 2 == 0:
                print("\nHmm...")
                time.sleep(1)
                try:
                  getSynonym(round)
                except:
                  get_example_sentence(word)
                chance_list.remove(2)
                chance_list.remove(4)
                time.sleep(1)
                print("\nDüşünün biraz daha...")

              elif chance_num % 7 == 0:
                print("\nHmm... Belki bu yardımcı olabilir...")
                time.sleep(1)
                print("\n",Example_sentences[round])
                time.sleep(2)
                print("Ayrıca...")
                try:
                  getSynonym(round)  
                except:
                  get_example_sentence(word)
                chance_list.remove(7)
                print("\nOdaklanırsanız bulursunuz bence... Tekrar okuyun...")

        elif ("eş" in tkn_input and "anlamlı" in tkn_input) or ("eş" in tkn_input and "anlamlısı" in tkn_input) or ("eş" in tkn_input and "anlam" in tkn_input) or ("benzer" in tkn_input and "anlamda" in tkn_input) or ("eş" in tkn_input and "anlamı" in tkn_input) or ("benzer" in tkn_input and "anlamlı" in tkn_input):
          if getSynonym(round) != "None":
            print("\n",getSynonym(round))
          else:
            print("\nMaalesef aklıma bir şey gelmedi şu an...")
                    
            
        else:
            string_familiarity = [similar_word_hint(word, 3)]
            list_synonym = [synonym_hint(word)]
            #return string_familiarity
            #return list_synonym

            if active_input.lower().strip() not in list_active_input:
              string_familiarity = similar_word_hint(word, 3)
              list_synonym = synonym_hint(word)
                
            if active_input.lower().strip() in string_familiarity and active_input.lower().strip() in list_synonym:
              print("\nHadi bir daha, çok çok yaklaştınız...")

            elif active_input.lower().strip() in string_familiarity or any(item in string_familiarity for item in tkn_input):
              print("\nÇok yaklaştınız, hadi birkaç harf değiştirin.")
            
            elif active_input.lower() == "":
              phrases_list_empty = ["Bir harf alabilirsiniz belki...", "...", "Zamana dikkat!", "Zaman akıyor efendim, bir harf mi istesek..?", "..."]
              print("\n",phrases_list_empty[random.randint(0, (len(phrases_list_else)-1))])

            elif active_input.lower().strip() in list_synonym or any(item in list_synonym for item in tkn_input):
              synonym_phrases = ["Bu değil efendim, öteki... Eş anlamlısı lazım bize...", "Hadi bir daha deneyin... Bu sefer eş anlamlısını söyleyin...", f"{active_input} değil de... Bir benzeri...\nNeydi o?\nBize eş anlamlısı lazım...", "Yaklaştınız... Aynı anlama gelen başka kelime daha var...\nNeydi o..?"]
              print("\n",synonym_phrases[random.randint(0, (len(synonym_phrases) - 1))])

            elif active_input.lower().strip() in list_active_input or any(item in list_active_input for item in tkn_input):
              print("\nBunu zaten söylediniz, tekrar düşünün...")        

            else: 

              if editdistance.distance(active_input.lower().strip(), word) <= 2:
                if round >= 6:
                  distance_phrases = ["Çok yaklaştınız, acaba bir iki harf mi değiştirsek..?", f"{active_input.lower()} doğru muydu yoksa birkaç harf mi farklıydı?"] 
                  print(distance_phrases[random.randint(0, len(distance_phrases) - 1)]) 

              elif "h" in list_active_input:
                phrases_list_else = ["Bir harf daha alabilirsiniz belki...", "Biraz daha düşünün isterseniz...", "Zamana dikkat!", "Zaman akıyor efendim, bir harf daha mı istesek..?"]
                print("\n",phrases_list_else[random.randint(0, (len(phrases_list_else)-1))])
              else:
                phrases_list_else = ["Harf mi alsanız..?", "Biraz düşünün isterseniz...", "Zamana dikkat!", "Zaman akıyor efendim, harf mi istesek..?", "Hayır, hayır... Dikkatlice tekrar okuyun isterseniz soruyu..."]
                print("\n",phrases_list_else[random.randint(0, (len(phrases_list_else)-1))])

        list_active_input.append(active_input)
      chance_list = [2, 3, 4, 7, 9, 5, 25, 65] 

  # Initialize global variables
  total_time = 5 * 60  # 5 minutes in seconds
  global is_paused
  is_paused = True
  resume_time = total_time

  # Run countdown timer and take continuous input concurrently
  timer_thread = threading.Thread(target=countdown_timer)
  timer_thread.start()

  for round in range(14):
    revealed_letters = []
    if round != 0:
      ready = input("\nSıradaki soru ile devam etmek için \"d\" yazınız\nToplam skorunuzu görmek için \"puan\" yazınız: ")
      if ready.lower().strip() == "puan":      
        print(f"\nŞuanki toplam puanınız: {total_score} ")       
        ready = input("\nSıradaki soru ile devam etmek için \"d\" yazınız: ")

        if ready.lower().strip() == "d":     
          question_number = round+1

          global word
          word = word_list[round]

          Pre_info(question_number, word, total_score, round)

          time.sleep(2.3)
          letter_count = (((question_number-1) // 2) + 4)
          print("")

          print("_  "*letter_count, "\n")

          print(definition_list[round]) 

          is_paused = False
          Process()
          is_paused = True
      elif ready.lower().strip() == "d":     
        question_number = round+1

        
        word = word_list[round]

        Pre_info(question_number, word, total_score, round)

        time.sleep(2.3)
        letter_count = (((question_number-1) // 2) + 4)
        print("")

        print("_  "*letter_count, "\n")

        print(definition_list[round]) 

        is_paused = False
        Process()
        is_paused = True          
    else:
      question_number = round+1
      word = word_list[round]
      Pre_info(question_number, word, total_score, round)
      time.sleep(3)
      is_paused = False
      letter_count = (((question_number-1) // 2) + 4)
      print("")
      print("_  "*letter_count, "\n")
      print(definition_list[round]) 
      is_paused = False
      Process()
      is_paused = True
    revealed_letters = []

  # Wait for the timer thread to finish
  timer_thread.join()


In [74]:
from IPython.display import display, HTML

def redirect_to_survey():
    survey_link = "https://forms.gle/xhRYD5doZLKxumAw7"  # Replace with your Google Survey link
    redirect_html = f'<a href="{survey_link}" target="_blank">Lütfen deneyiminiz değerlendirmek için buraya tıklayın ve anketi doldurun :) </a>'
    display(HTML(redirect_html))

# Call the function to redirect the user


In [65]:

def run():
  Head()
  StartCommand()
  if start_com.strip() in ["Ve kayıt","ve kayıt"]:
    Gpt_Prologue()
    time.sleep(1)


    total_score = 0


    __main__x()
    time.sleep(3)
    end_phrases_super = [f"Tebrik ederim efendim yarışmayı {total_score} gibi harika bir puanla tamamladınız...", f"Bu sorudan sonra kasamız {total_score} puana çıkıyor ve {username} yarışmayı bu müthiş puanla bitiriyor...\nTebrikler efendim..." ]
    end_phrases_adequate = [f"Sizi tebrik ederim efendim... Belli ki bugün zor sorular denk gelmiş...\nYine de yarışmayı {total_score} gibi güzel bir puanla tamamladınız...", f"{total_score} puan... Bu zor sorular için gayet iyi bir sonuçla yarışmayı tamamladınız...\nTebrik ederim..."]
    end_phrases_low = [f"Bu son soru ile birlikte puanımızı {total_score} puana çıkarttık...\nBelki hedeflediğimiz değildi ancak üzülmeyin tekrar denersiniz...\n\nKapımız size her zaman açık...", f"Bu soru ile yarışmayı tamamladık...\nBelki arzu ettiğimiz puanlara ulaşamadık ancak {total_score} puanın suçunu basiret bağlanmasına atabiliriz\nve sizi aramızda tekrar görmeyi çok isteriz..."]

    if total_score >= 8500:
      print(end_phrases_super[random.randint(0, len(end_phrases_super) - 1)]) 
    elif total_score >= 7000:
      print(end_phrases_adequate[random.randint(0, len(end_phrases_adequate) - 1)]) 
    else:
      print(end_phrases_low[random.randint(0, len(end_phrases_low) - 1)])  

    time.sleep(4)
    print("\nTekrar görüşmek üzere...\n\nSağlıcakla kalın bizi de öksüz bırakmayın...")
    redirect_to_survey()

In [ ]:
# After running every cell run this line, uncomment the code (remove "#"").

#run()
